## Drug Repurposing: A Hybrid Graph Convolutional Network for Predicting Cancer Drug Response

The goal of this pipeline is to implement a Cancer Drug Response Prediction pipeline. The model chosen is called DeepCDR which is a Cancer Drug Response Prediction model via a Hybrid Graph Convolutional Network. This model was published in the journal Bioinformatics.

<center><img src="./images/model.png" style="width:70%;"></center>

DeepCDR takes a pair of drug and cancer cell profiles as inputs and predicts the drug sensitivity (IC50) (regression). The drug will be represented as a graph based on the chemical structure before being transformed into a high-level latent representation by the GNN model. Omics features learned by subnetworks will be concatenated to the drug feature.

## README

The [README](README.md) file accompanying this notebook includes additional information about the research behind this model and how to run this pipeline with your own data. 

## Katana AI Pipeline for End to End Drug Repurposing

<img src="images/pipeline_full.png" style="width: 1500px;"/>



There are four distinct steps:

* **[Step 1: Set up a Katana `Client` and create a Distributed Graph](#step-1)**
* **[Step 2: Extract ML-ready Data](#step-2)**
* **[Step 3: AI Training Pipeline](#step-3)**
* **[Step 4: Trained model inference](#step-4)**

<a id='step-1'></a>
## Step 1: Set up a Katana `Client` and load a Distributed Graph

<img src="images/pipeline_dask.png" style="width: 1500px;"/>

### Set up a Katana `Client`

Starting a Katana remote Client is required to interface with the Katana remote service and schedule distributed operations. First we set Jupyter to automatically reload changed code and we import some standard Katana modules.

In [4]:

%load_ext autoreload
%autoreload 2

from katana import remote
from katana.remote import export_data, import_data


client = remote.Client()
print()
print("Software version: %s" % (client.server_version))

print("--")


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
Software version: 0.9.0+20230322T224759Z.73f024927.dev
--


In [11]:

#  An environment variable with the given titles affects execution ..
#

import os


del os.environ['RED_SHORT_RECIPE'] 
   #
l_env = os.environ.get('RED_SHORT_RECIPE')
print("RED_SHORT_RECIPE Set to: %s" % (l_env))


os.environ['RED_SHORT_RECIPE'] = "1"
   #
l_env = os.environ.get('RED_SHORT_RECIPE')
print("RED_SHORT_RECIPE Set to: %s" % (l_env))

print("--")


RED_SHORT_RECIPE Set to: None
RED_SHORT_RECIPE Set to: 1
--


### Load Input configuration

The input configuration, which is stored in the `hyperparams` file, defines the specifications of the graph use for the recipe.

In [ ]:
from config import hyperparams

input_config = hyperparams.load_input_config()
input_config

### Create a Distributed Graph

Create a Katana Distributed graph with `input_config.num_partitions` partitions and import data used to train the DeepCDR model into the Distributed Graph. The entire recipe will be distributed among `input_config.num_partitions` hosts.

<div class="alert alert-info">
You can choose to change the number of partition using <a href="./config/hyperparams.py" class="alert-link">num_partitions</a> parameter.
</div>

<div class="alert alert-info">
You can choose to import a graph with the features already generated and a model already trained and saved in the graph or create your graph from csv files with Dask by setting <a href="./config/hyperparams.py" class="alert-link">use_train_rdg</a> parameter. 
    
If you choose to use the pretrained model you can skip the following two steps,
* **[Step 2: Extract ML-ready Data](#step-2)**
* **[Step 3: AI Training Pipeline](#step-3)**

and go directly to:
* **[Step 4: Trained model inference](#step-4)**
</div>

In [ ]:
from timeit import default_timer

# Import the module that uses Dask to import drug data
from src import dask_ingestion

graph = client.create_graph(num_partitions=input_config.num_partitions)

if input_config.use_train_rdg:
    print(f"Import pretrained graph from: {input_config.trained_rdg_path}")
    import_data.rdg(graph, input_config.trained_rdg_path)
else:
    print("Generate the graph with data from source")
    dask_ingestion.generate_deepcdr_graph(graph, input_config)

### Inspect the Graph Schema

Import the module with pipeline utility functions, create a `RecipePipeline` instance to store relevant information, and display a visual representation of the graph's schema.

In [ ]:
from src import katana_pipeline

rec_pipeline = katana_pipeline.RecipePipeline(graph)
rec_pipeline.graph.schema().view()

<a id='step-2'></a>
## Step 2: Extract ML-ready Data

<img src="images/pipeline_preprocess.png" style="width: 1500px;"/>

### Feature Generation

- Collect genomics data from edges between `CELL_LINE` and `GENE` and store the results in the `rec_pipeline` graph. 
- Remove unused nodes in the graph. 


<div class="alert alert-info">
You can see, add or remove preprocessing functions in the <a href="./src/preprocessing.py" class="alert-link">preprocessing.py</a> file, where they are typically implemented with simple OpenCypher queries.
</div>

The featurization of a `DRUG` into its graph representation is happening on-the-fly during the training-test-inference cycle. 

<div class="alert alert-info">
You can see or change the featurization of DRUG in the smiles_to_pyg function of the <a href="./src/utils.py" class="alert-link">utils.py</a> file. 
</div>

In [ ]:
start_time = default_timer()
rec_pipeline.feature_generator()
print(f"***Took {default_timer() - start_time} seconds to generate the features.***")

### Statistics of the graph

Compute basics statistics to get the number of nodes in the graph. The number of pairs between a drug and a cell line is the data used to train and test the model.

In [ ]:
stats = rec_pipeline.stats()
stats

### Split Generation

Based on the known drug–cell line interactions across cancer cell lines  and drugs, we randomly
selected 95% of instances of each TCGA cancer type as the training set and the remaining 5% of the instances as the testing set for model evaluation.

We create new edges in our `rec_pipeline` graph with `SPLIT` labels between `DRUG` and `CELL_LINE` nodes. The old split will be deleted.

<div class="alert alert-info">
You can choose to change the train-test fraction or the seeds used for the split using the <a href="./config/hyperparams.py" class="alert-link">test_size</a> and <a href="./config/hyperparams.py" class="alert-link">random_state</a> parameters.
</div>

In [ ]:
start_time = default_timer()
rec_pipeline.split_generator(input_config)
print(f"***Took {default_timer() - start_time} seconds to generate the split.***")
rec_pipeline.graph.schema().view()

<a id='step-3'></a>
## AI Training Pipeline

<img src="images/pipeline_gnn.png" style="width: 1500px;"/>

### Initialize model training

<center><img src="./images/model.png" style="width:70%;"></center>

DeepCDR is a hybrid graph convolutional network model that predicts the IC50 value, which denotes the effectiveness of a drug in inhibiting the growth of a specific cancer cell line.

On the one hand, the graph neural network (GNN) takes the adjacent information of atoms in a drug into consideration by aggregating the features of neighboring atoms together. On the other hand, the subnetworks extract high-level features of cancer omics profiles from a certain cancer cell line (i.e., genomic data, transcriptomic data and epigenomic data). Then the high-level features of drug and multiple omics data get concatenated and fed to a 1-D
convolutional neural network.

### Load Model configuration

The model configuration defines the specifications of the model that will be used to train on the graph.

<div class="alert alert-info">
You can change the model configuration in the <a href="./config/hyperparams.py" class="alert-link">hyperparams.py</a> file. By default the model is using genomics, epigenomics and transcriptomics data.
</div>

<div class="alert alert-info">
You can change the model architecture in the <a href="./src/model.py" class="alert-link">model.py</a> file.
</div>

In [ ]:
model_config = hyperparams.load_model_config()
model_config

### Load Training configuration

The training configuration specify details of the training pipeline.

<div class="alert alert-info">
You can change the training configuration in the <a href="./config/hyperparams.py" class="alert-link">hyperparams.py</a> file.
</div>

In [ ]:
training_config = hyperparams.load_training_config()
training_config

### Training

Distributed training of the model. The trained model and its configuration will be saved in the `rec_pipeline` graph using the `experiment_name` graph property.

In [ ]:
start_time = default_timer()
validation_metric = rec_pipeline.train(model_config, training_config)
print(f"***Took {default_timer() - start_time} seconds to train the model.***")
print("Validation metric: ", validation_metric)

### Testing

Test the trained model on the test data and compare the results with other achitecture baselines. We compare three different metrics:
- [Pearson correlation coefficient](https://en.wikipedia.org/wiki/Pearson_correlation_coefficient): a measure of linear correlation between two sets of data. A value closer to 1 is better.
- [Spearman correlation coefficient](https://en.wikipedia.org/wiki/Spearman%27s_rank_correlation_coefficient): how well the relationship between two variables can be described using a monotonic function. A value closer to 1 is better.
- [Root mean square error (RMSE)](https://en.wikipedia.org/wiki/Root-mean-square_deviation): a measure of the differences between values predicted by a model and the values observed. A lower value is better.

In [ ]:
start_time = default_timer()
metrics, predictions, labels = rec_pipeline.test(training_config)
print(f"***Took {default_timer() - start_time} seconds to test the model.***")
metrics

### Plot Predictions

Create a scatter plot of the `IC50` predicted by the trained model in function of `IC50` real value.

In [ ]:
start_time = default_timer()
rec_pipeline.plot(labels, predictions)
print(f"***Took {default_timer() - start_time} seconds to plot figures.***")

<a id='step-4'></a>
## Trained model inference 

<img src="images/pipeline_inference.png" style="width: 1500px;"/>



In [ ]:
rec_pipeline.infer(training_config)

In [ ]:
bortezomib = "B(C(CC(C)C)NC(=O)C(CC1=CC=CC=C1)NC(=O)C2=NC=CN=C2)(O)O"
cell_line = "ACH-000001"
rec_pipeline.infer(training_config, drug=bortezomib, cell_line=cell_line)

## Run trained model to save node embeddings

Save the trained model embeddings as a node property. `drug_embeddings`, `epigenomics_embeddings`, `genomics_embeddings` and `transcriptomics_embeddings` will be created and saved. Those embeddings can be use for a future downstream task with a different model.

In [ ]:
start_time = default_timer()
rec_pipeline.infer_embeddings(model_config)
print(f"***Took {default_timer() - start_time} seconds to save node embeddings.***")
rec_pipeline.graph.schema().view()

## Save Graph

Save the graph created to the bucket location named in the `save_graph_path` variable.

In [ ]:
if input_config.save_graph_path:
    export_data.rdg(graph, input_config.save_graph_path)